# 输出销量的部分预测

1. 过去的销量
2. 过去的生产、库存
3. （对于未来的预测）共同决定未来的生产

In [1]:
from imports import *

In [2]:
# read order details from GE_DIR

order = pd.read_csv(os.path.join(GE_DIR, 'order_detail.csv'))

In [3]:
order

,id,date,time,datetime,provice,city
0,0,2022-12-01,22:45,2022-12-01 22:45,北京,北京市
1,1,2022-12-01,13:47,2022-12-01 13:47,江苏,苏州市
2,2,2022-12-01,10:39,2022-12-01 10:39,江苏,扬州市
3,3,2022-12-01,10:40,2022-12-01 10:40,江苏,淮安市(原淮阴市）
4,4,2022-12-01,15:43,2022-12-01 15:43,上海,上海市
...,...,...,...,...,...,...
3539,106,2022-12-31,09:44,2022-12-31 09:44,上海,上海市
3540,107,2022-12-31,13:49,2022-12-31 13:49,上海,上海市
3541,108,2022-12-31,14:35,2022-12-31 14:35,江苏,泰州市
3542,109,2022-12-31,08:04,2022-12-31 08:04,浙江,温州市


In [4]:
# groupby date, provice
# df = order.groupby(['date', 'provice']).count().reset_index()
df_by_date = order.groupby('date').count().reset_index()
df_by_date

,date,id,time,datetime,provice,city
0,2022-12-01,46,46,46,46,46
1,2022-12-02,41,41,41,41,41
2,2022-12-03,81,81,81,81,81
3,2022-12-04,118,118,118,118,118
4,2022-12-05,176,176,176,176,176
5,2022-12-06,113,113,113,113,113
6,2022-12-07,197,197,197,197,197
7,2022-12-08,28,28,28,28,28
8,2022-12-09,35,35,35,35,35
9,2022-12-10,105,105,105,105,105


In [5]:
px.line(df_by_date, x='date', y='id', color='provice')

# Prediciton with ARIMA

In [6]:

# extend dates
def extend_dates(last_date, n = 30):
    
    # last_date = dates[-1]
    # next day
    x = [datetime.strptime(last_date, '%Y-%m-%d') + timedelta(days=i) for i in range(n)]
    x = [datetime.strftime(x,'%Y-%m-%d') for x in x]
        
    return x

In [7]:
# Use ARIMA model to predict order count

from statsmodels.tsa.arima.model import ARIMA

# df_for_pred = df[df['provice'] == '上海'].reset_index(drop=True)
df_for_pred = df_by_date


def pred(df_for_pred):
    # use data from 上海
    data = df_for_pred['id'].to_list()
    train = data[:-10]
    test = data[-10:]
    # train, test
    model = ARIMA(train, order=(2,1,2)).fit()  # ARIMA = (2,1,2)

    delta = model.fittedvalues - train

    # score = 1 - delta.var()/train[1:].var()
    pred = model.predict(start=1, end=len(train)+10)
    
    # confidence 
    conf = model.conf_int(alpha=0.05)
    
    df_data = pd.DataFrame()
    df_data['date'] = df_for_pred['date']
    df_data['value'] = data
    df_data['source'] = 'History'
    df_data

    # extend dates

    dates = df_data['date'].values
    last_date = dates[-1]
    
    df_pred = pd.DataFrame()
    df_pred['date'] = extend_dates(last_date, len(pred))
    df_pred['value'] = pred
    df_pred['source'] = 'Prediction'
    df_all = pd.concat([df_data, df_pred], axis=0)
    
    return df_all, model

In [9]:
df_all, model = pred(df_for_pred)

/Users/zhitaogao/opt/anaconda3/envs/mfg/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

/Users/zhitaogao/opt/anaconda3/envs/mfg/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.



In [44]:
forecast = model.get_forecast(30)
yhat = forecast.predicted_mean
yhat_conf_int = forecast.conf_int(alpha=0.1)

In [45]:
# change to dataframe
yhat_conf_int_df = pd.DataFrame(yhat_conf_int)

In [46]:
yhat_conf_int_df

,0,1
0,40.375015,231.746451
1,14.241486,237.708052
2,5.955542,231.793909
3,3.526684,232.934718
4,1.459459,230.540669
5,1.595265,231.773535
6,0.734686,230.508651
7,1.125209,231.466389
8,0.638562,230.656235
9,0.963331,231.334309


In [47]:
yhat_conf_int_df.columns = ['lower', 'upper']

In [48]:
# plot
fig = px.line(df_all, x='date', y='value', color='source')

fig.add_trace(go.Scatter(x=df_all['date'], y=yhat, name='Prediction_mean'))
fig.add_trace(go.Scatter(x=df_all['date'], y=yhat_conf_int_df['lower'], name='Lower'))
fig.add_trace(go.Scatter(x=df_all['date'], y=yhat_conf_int_df['upper'], name='Upper'))

In [16]:
# plot the result

px.line(df_all, x='date', y='value', color='source')

In [17]:
# Save the result as csv to GE_DIR
df_all.to_csv(os.path.join(GE_DIR, 'order_prediction.csv'), index=False)

# Prediction with Prophet  (Not Use for now)

In [40]:
# !pip install prophet
# !pip install fbprophet

In [37]:
from prophet import Prophet

In [38]:
# fit prophet model
model = Prophet()

In [41]:
model.fit()

TypeError: Prophet.fit() missing 1 required positional argument: 'df'